In [1]:
import numpy as np
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
Base = automap_base()

In [7]:
Base.prepare(engine, reflect=True)

In [8]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
session = Session(engine)
app = Flask(__name__)

In [10]:
@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/start/end"
    )

In [11]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    session = Session(engine)

    results = session.query(Measurement.date, Measurement.prcp).all()

    session.close()

    all_precipitation = []
    for date, prcp in results:
        precipitation_dict = {date : prcp}
        precipitation_dict["date"] = date
        precipitation_dict["prcp"] = prcp
        all_precipitation.append(precipitation_dict)

    return jsonify(all_precipitation)

In [12]:
@app.route("/api/v1.0/stations")
def stations():
    session = Session(engine)

    # Query all stations
    results = session.query(Station.station).all()

    session.close()
    
    all_stations = list(np.ravel(results))

    return jsonify(all_stations)

In [13]:
@app.route("/api/v1.0/tobs")
def tobs():
    # Create our session (link) from Python to the DB
    session = Session(engine)

    most_active_station = session.query(Measurement.station, func.count(Measurement.tobs)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.tobs).desc()).first()

    session.close()
    
    full_year = session.query(Measurement.date, Measurement.station, Measurement.tobs).\
    filter(Measurement.date >= '2016-08-24').\
    filter(Measurement.station == most_active_station[0]).\
    group_by(Measurement.date).\
    order_by(Measurement.date.desc()).all()

    return jsonify(full_year)

In [14]:
@app.route("/api/v1.0/start/end")
def start_end_():
    session = Session(engine)

    start = '2017-07-04'
    end = '2017-07-11'

    start_end_observations = session.query(Measurement.station, Measurement.date, func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.date >= start).\
    filter(Measurement.date <= end).\
    group_by(Measurement.station).\
    order_by(Measurement.date.desc()).all()
    
    session.close()

    return jsonify(start_end_observations)


In [16]:
if __name__ == '__main__':
    app.run(debug=True)
    
